### External validation 

DATA: NKI

1) Read external data set
2) Include Thresholds 
3) Preprocess external data set as in next_assessment_dynamic_preprocessing.py
4) Validate model

In [ ]:
import pandas as pd
import numpy as np 

# Load the external validation dataset
df = pd.read_csv('data/nki_validation.csv', sep=",", decimal=",")

In [ ]:
import pandas as pd
import numpy as np

cols_to_convert = [
    "C30_PF2", "C30_RF2", "C30_SF", "C30_EF", "C30_CF", 
    "C30_FA", "C30_PA", "C30_NV", "C30_SL", "C30_DY", 
    "C30_AP", "C30_CO", "C30_DI", "C30_FI"
]

df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors="coerce")

# Apply conditions after conversion
df1 = df.assign(
    C30_PF2_class=np.where(df['C30_PF2'] < 83, 1, 0),  # Physical functioning
    C30_RF2_class=np.where(df['C30_RF2'] < 58, 1, 0),  # Role functioning
    C30_SF_class=np.where(df['C30_SF'] < 58, 1, 0),    # Social functioning
    C30_EF_class=np.where(df['C30_EF'] < 71, 1, 0),    # Emotional functioning
    C30_CF_class=np.where(df['C30_CF'] < 75, 1, 0),    # Cognitive functioning
    C30_FA_class=np.where(df['C30_FA'] > 39, 1, 0),    # Fatigue
    C30_PA_class=np.where(df['C30_PA'] > 25, 1, 0),    # Pain
    C30_NV_class=np.where(df['C30_NV'] > 8, 1, 0),     # Nausea
    C30_SL_class=np.where(df['C30_SL'] > 50, 1, 0),    # Sleep disturbances
    C30_DY_class=np.where(df['C30_DY'] > 17, 1, 0),    # Dyspnea
    C30_AP_class=np.where(df['C30_AP'] > 50, 1, 0),    # Appetite loss
    C30_CO_class=np.where(df['C30_CO'] > 50, 1, 0),    # Constipation
    C30_DI_class=np.where(df['C30_DI'] > 17, 1, 0),    # Diarrhea
    C30_FI_class=np.where(df['C30_FI'] > 17, 1, 0)     # Financial impact
)

df1.columns.values

In [ ]:
# drop rows without assessment date
df2 = df1.dropna(subset=['Assessment_date_days'])
df2

In [ ]:
# drop all patients that have only one assessment date
assessment_counts = df2.groupby('BALANCE_ID').count()['Pat_num']
patients_with_two_assessments = assessment_counts[assessment_counts > 1].index.values
df3 = df2[df2.BALANCE_ID.isin(patients_with_two_assessments)]
df3

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoded = [
        "Marital_status",
        "Education_status",
        "Diag_code",
        "Lateralisation",
        "Location_code",
        "Histology_code",
        "Differentation_grade",
        'Country',
        'T_staging',
        'N_staging',
        "Stage",
        "Her2Neu_status",
        "T_staging_postOP",
        "N_staging_postOP",
        "Stage_postOP",
        'Treatment_1', 'Treatment_2', 'Treatment_3', 'Treatment_4', 'Treatment_5', 'Treatment_6', 
        'Treatment_7', 'Treatment_8', 'Treatment_9', 'Treatment_10', 'Treatment_11', 'Treatment_12'
]


ohe = OneHotEncoder()
raw_data = ohe.fit_transform(df3[one_hot_encoded]).todense()
one_hot_column_names = ohe.get_feature_names_out(one_hot_encoded)
df_one_hot = pd.DataFrame(raw_data, columns=one_hot_column_names)
df_one_hot.index = df3.index
df4 = pd.concat([df3.drop(columns=one_hot_encoded), df_one_hot], axis=1)
df4['Source'] = 2
df4.columns = df4.columns.str.replace(r'\.0$', '', regex=True)
df4

In [ ]:
df_all_columns = pd.read_pickle('data/cached_df.pckl')
df5 = pd.concat([df_all_columns, df4])
df5 = df5[df5.Source == 2]

In [ ]:
from constants import targets, get_feature_sets

feature_sets = get_feature_sets(df5)
# Sort the data to ensure temporal ordering within each BALANCE_ID
validation_data = df5.dropna(subset=targets).sort_values(by=['BALANCE_ID', 'Assessment_date_days'])

Xs, ys = [], []
lastrow = None

patient_ids = df5.BALANCE_ID.unique()
for patient_id in patient_ids:
    pdf = df5[df5.BALANCE_ID == patient_id]
    for i in range(0, pdf.shape[0] - 1):
        for j in range(i + 1, pdf.shape[0]):
            assessment0 = pdf.iloc[i].copy()
            assessment1 = pdf.iloc[j]

            time_diff = assessment1['Assessment_date_days'] - assessment0['Assessment_date_days']
            assessment0['time_diff'] = time_diff
            Xs.append(assessment0)
            ys.append(assessment1[['BALANCE_ID'] + targets])
            
# Convert to DataFrames
X = pd.DataFrame(Xs).reset_index(drop=True)
y = pd.DataFrame(ys).reset_index(drop=True)            

In [ ]:
X.to_pickle('data/external_x.pckl')
y.to_pickle('data/external_y.pckl')

y.to_csv('data/targets_external.csv')
X.to_csv('data/predictors_external.csv')

In [ ]:
print(X["BALANCE_ID"].nunique())
print(X.shape)

In [ ]:
assert X.shape[0] == y.shape[0]

In [ ]:
import numpy as np

# Extract features and ensure only the required features are used
# Ensure only the features that exist in the validation_data are selected
valid_features = [feature for feature in feature_sets['all'] if feature in validation_data.columns]
valid_features.append('Source')
features = X[valid_features]  # Select only valid features

In [ ]:
import pickle
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import resample
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import (
    roc_auc_score, f1_score, accuracy_score, balanced_accuracy_score, 
    recall_score, average_precision_score, brier_score_loss
)

n_bootstraps = 1000
rng = np.random.RandomState(42)

# Initialize a dictionary for the results
results = {}

with PdfPages("calibration_plots.pdf") as pdf:
    # Iterate through each target in the targets list
    for target in targets:
    
        # Load the model from the file
        with open(f'calibrated_model_{target}.pkl', 'rb') as file:
            model = pickle.load(file)
        
        print(f"Validating for target: {target}")
    
        # Ensure the target column exists in the validation dataset
        if target not in validation_data.columns:
            print(f"Warning: Target '{target}' not found in the dataset. Skipping...\n")
            continue
    
        y_true = y[target]  # Extract the target column
    
        # Validate the model
        y_pred = model.predict(features)
        
        y_proba = model.predict_proba(features)[:, 1]  # Probability for the positive class
    
        boot_metrics = {
            'roc_auc': [],
            'f1': [],
            'f1_weighted': [],
            'accuracy': [],
            'balanced_accuracy': [],
            'recall_weighted': [],
            'average_precision': [],
            'brier_score_loss': [],
            'calibration_slope': [],
            'calibration_intercept': []
        }
        
        # Initialize a dictionary for this target
        results[target] = {}
    
        for i in range(n_bootstraps):
            indices = rng.choice(range(len(y_true)), size=len(y_true), replace=True)
            y_true_bs = y_true.iloc[indices]
            y_pred_bs = y_pred[indices]
            y_proba_bs = y_proba[indices]
        
            try:
                boot_metrics['roc_auc'].append(roc_auc_score(y_true_bs, y_proba_bs))
                boot_metrics['f1'].append(f1_score(y_true_bs, y_pred_bs))
                boot_metrics['f1_weighted'].append(f1_score(y_true_bs, y_pred_bs, average='weighted'))
                boot_metrics['accuracy'].append(accuracy_score(y_true_bs, y_pred_bs))
                boot_metrics['balanced_accuracy'].append(balanced_accuracy_score(y_true_bs, y_pred_bs))
                boot_metrics['recall_weighted'].append(recall_score(y_true_bs, y_pred_bs, average='weighted'))
                boot_metrics['average_precision'].append(average_precision_score(y_true_bs, y_proba_bs))
                boot_metrics['brier_score_loss'].append(brier_score_loss(y_true_bs, y_proba_bs))
                
                # Compute calibration slope and intercept:
                # Clip probabilities to avoid numeric issues with the logit transformation
                y_proba_bs_clipped = np.clip(y_proba_bs, 1e-15, 1-1e-15)
                logits = np.log(y_proba_bs_clipped / (1 - y_proba_bs_clipped))
                # Use linear regression (np.polyfit returns [slope, intercept] when degree=1)
                slope, intercept = np.polyfit(logits, y_true_bs, 1)
                boot_metrics['calibration_slope'].append(slope)
                boot_metrics['calibration_intercept'].append(intercept)
                
            except ValueError:
                # Happens if a bootstrap sample contains only one class
                continue
    
        def ci_bounds(metric_list):
            return np.percentile(metric_list, [2.5, 97.5])
        
        results[target]['External Dataset (NKI)'] = {}
        for metric, scores in boot_metrics.items():
            if len(scores) > 0:
                mean_val = np.mean(scores)
                ci_low, ci_high = ci_bounds(scores)
                results[target]['External Dataset (NKI)'][metric] = {
                    'mean': mean_val,
                    '95% CI': (ci_low, ci_high)
                }
    
        # Compute calibration curve
        prob_true, prob_pred = calibration_curve(y_true, y_proba, n_bins=10)
    
        # Plot calibration curve 
        fig, ax = plt.subplots(figsize=(7, 6))
        ax.plot(prob_pred, prob_true, marker='o', label="Model Calibration")
        ax.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Perfect Calibration")
        ax.set_xlabel("Mean Predicted Probability")
        ax.set_ylabel("Fraction of Positives")
        ax.set_title(f"Calibration Plot: {target}")
        ax.legend()
        ax.grid(True)
        
        plt.tight_layout()
        plt.show()

        # Save the current figure into the PDF
        pdf.savefig(fig)
        plt.close(fig)

In [ ]:
# Create result export

rows = []

for target, classifiers in results.items():
    for classifier_name, metrics in classifiers.items():
        row = {
            'Target': target,
            'Classifier': classifier_name
        }

        for metric_name, value in metrics.items():
            if isinstance(value, dict) and 'mean' in value and '95% CI' in value:
                # It's a nested dictionary with mean and confidence interval
                row[f"{metric_name}"] = round(value['mean'], 3)
                row[f"{metric_name}_ci_low"] = round(value['95% CI'][0], 3)
                row[f"{metric_name}_ci_high"] = round(value['95% CI'][1], 3)
            elif isinstance(value, list) and len(value) == 1:
                # Single value stored as a list
                row[metric_name] = round(value[0], 3)
            else:
                # Fallback for unexpected structure
                row[metric_name] = value

        rows.append(row)

# Create DataFrame
results_df = pd.DataFrame(rows)

# Display
display(results_df)

# Save to CSV
results_df.to_csv('results_external_validation.csv', index=False)